In [1]:
from google.colab import drive
drive.mount("/content/gdrive", force_remount=True)

Mounted at /content/gdrive


In [2]:
%cd /content/gdrive/MyDrive/Datasets/Apple_leaf_dataset/

/content/gdrive/MyDrive/Datasets/Apple_leaf_dataset


In [3]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/MyDrive/Datasets/Apple_leaf_dataset/.kaggle"

In [14]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
import shutil
import random
from PIL import Image
import matplotlib.image as mpimg
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, MaxPooling2D, Flatten, Dropout, Conv2D, Activation, BatchNormalization
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [5]:
df = pd.read_csv('/content/gdrive/MyDrive/Datasets/Apple_leaf_dataset/train.csv', index_col = 0)
df.head()

,healthy,multiple_diseases,rust,scab
image_id,,,,
Train_0,0,0,0,1
Train_1,0,1,0,0
Train_2,1,0,0,0
Train_3,0,0,1,0
Train_4,1,0,0,0


In [6]:
img_size = 128
images = []
labels = []
IMAGE_DIR = '/content/gdrive/MyDrive/Datasets/Apple_leaf_dataset/images'
for index, values in df.iterrows():
    get_image_path = os.path.join(IMAGE_DIR, index + '.jpg')
    #target_path = os.path.join(TRAIN_DIR, label,index + '.jpg')
    #shutil.copy(get_image_path, target_path)
    img = cv2.imread(get_image_path)
    img = cv2.resize(img, (img_size, img_size))
    img = img/255
    label = [values]
    images.append(img)
    labels.append(label)


In [7]:
img_data_g = tf.keras.preprocessing.image.ImageDataGenerator(
        rotation_range=0.35,
        zoom_range=0.2,
        horizontal_flip=True,
        vertical_flip=True,
        shear_range=0.2,
        width_shift_range=0.15,
        height_shift_range=0.15,
        rescale=None)

In [8]:
images_after_aug = []
labels_after_aug = []
for index, image in enumerate(images):
  for i in range(9):
      img = img_data_g.flow(np.reshape(image, (1, 128, 128, 3))).next()
      images_after_aug.append(np.reshape(img, (128, 128, 3)))
      labels_after_aug.append(labels[index])
labels = np.asarray(labels_after_aug)
images = np.asarray(images_after_aug)
images.shape
labels.shape

(16389, 1, 4)

In [9]:
images.shape

(16389, 128, 128, 3)

In [10]:
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(images, labels, test_size = 0.2, random_state = 42)

In [11]:
!pip install hyperas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 121 kB 4.4 MB/s 
     |████████████████████████████████| 1.6 MB 31.2 MB/s 
     |████████████████████████████████| 83 kB 2.5 MB/s 


In [12]:
from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform

In [ ]:
def data():
    (X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=12345)
    X_train = X_train.astype('float32')
    X_val = X_val.astype('float32')
    X_train /= 255
    X_val /= 255
    nb_classes = 10
    Y_train = np_utils.to_categorical(y_train, nb_classes)
    Y_val = np_utils.to_categorical(y_val, nb_classes)
    return X_train, Y_train, X_val, Y_val

In [15]:
def model(x_train, y_train, x_val, y_val):
    
    model = Sequential()
    model_choice = {{choice(['one', 'two'])}}
    if model_choice == 'one':
        model.add(Conv2D(16, kernel_size=3, activation='relu',padding='same', input_shape=(128,128,3)))
        model.add(Conv2D(16, kernel_size=3, activation='relu',padding='same'))
        model.add(MaxPooling2D(pool_size=2,strides=2))
        model.add(Dropout({{uniform(0, 1)}}))
        model.add(Conv2D(32, kernel_size=3, activation='relu'))
        model.add(Conv2D(32, kernel_size=3, activation='relu'))
        model.add(BatchNormalization())
        model.add(MaxPooling2D(pool_size=2,strides=2))
        model.add(Dropout({{uniform(0, 1)}}))
    elif model_choice == 'two':
        model.add(Conv2D(32, kernel_size=3, activation='relu',padding='same', input_shape=(128,128,3)))
        model.add(Conv2D(32, kernel_size=3, activation='relu',padding='same'))
        model.add(MaxPooling2D(pool_size=2,strides=2))
        model.add(Dropout({{uniform(0, 1)}}))
        model.add(Conv2D(64, kernel_size=3, activation='relu'))
        model.add(Conv2D(64, kernel_size=3, activation='relu'))
        model.add(BatchNormalization())
        model.add(MaxPooling2D(pool_size=2,strides=2))
        model.add(Dropout({{uniform(0, 1)}}))
    
    model.add(Flatten())
    model.add(Dense({{choice([256, 512,1024])}}, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout({{uniform(0, 1)}}))
    choiceval = {{choice(['one', 'two'])}}
    if choiceval == 'two':
        model.add(Dense({{choice([256, 512,1024])}}, activation='relu'))
        model.add(BatchNormalization())
        model.add(Dropout({{uniform(0, 1)}}))
    
    model.add(Dense(10, activation='softmax'))
    
    adam = tf.keras.optimizers.Adam(lr=0.001)
    
    model.compile(loss='categorical_crossentropy', metrics=['accuracy'],
                  optimizer=adam)
    model.fit(x_train, y_train,
              batch_size=16,
              nb_epoch=35,
              verbose=2,
              validation_data=(x_val, y_val))
    score, acc = model.evaluate(x_val, y_val, verbose=0)
    print('Val accuracy:', acc)
    return {'loss': -acc, 'status': STATUS_OK, 'model': model}

In [23]:
data = (x_train, y_train, x_val, y_val)
best_run, best_model = optim.minimize(model=model,
                                      data= data,
                                      algo=tpe.suggest,
                                      max_evals=30,
                                      trials=Trials(),
                                      notebook_name='Apple_data2')


>>> Imports:
#coding=utf-8

try:
    from google.colab import drive
except:
    pass

try:
    import os
except:
    pass

try:
    import tensorflow as tf
except:
    pass

try:
    import pandas as pd
except:
    pass

try:
    import numpy as np
except:
    pass

try:
    import matplotlib.pyplot as plt
except:
    pass

try:
    import cv2
except:
    pass

try:
    import shutil
except:
    pass

try:
    import random
except:
    pass

try:
    from PIL import Image
except:
    pass

try:
    import matplotlib.image as mpimg
except:
    pass

try:
    from tensorflow.keras.preprocessing.image import ImageDataGenerator
except:
    pass

try:
    from tensorflow.keras.models import Sequential
except:
    pass

try:
    from tensorflow.keras.layers import Dense, MaxPooling2D, Flatten, Dropout, Conv2D, Activation, BatchNormalization
except:
    pass

try:
    from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
except:
    pass

try:
    from sklearn.model_selection 

TypeError: ignored